In [ ]:
#taken and modified from https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/fall2021/extra_topics/twitter_v2_example.ipynb

In [ ]:
import tweepy
import time
import re
import pandas as pd
bearer_token = 'xxxx'

In [ ]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
#Dates (days) stored in a excel file

deefe = pd.read_excel('DatesExcel.xlsx')
tiempo = [str(t)[:10] for t in deefe.keys()]

In [ ]:
#Crawling by setting hour limits, results stored in a json object with nested list form

fetched_t = []
for dates in tiempo:
    for response in tweepy.Paginator(client.search_all_tweets, 
                                     query = 'place_country:ES -is:retweet lang:es',
                                     user_fields = ['username', 'public_metrics', 'description', 'location'],
                                     tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                     expansions = 'author_id',
                                     start_time = dates + 'T10:45:00Z',
                                     end_time = dates + 'T20:00:00Z',
                                     max_results=500):
        
            time.sleep(5)
            fetched_t.append(response)

In [ ]:
#turning the the list into a dicctionary and a list, then into a Dataframe

result = []
user_dict = {}
# Loop through each response object
for response in fetched_t:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    try: 
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                  'followers': user.public_metrics['followers_count'],
                                  'tweets': user.public_metrics['tweet_count'],
                                  'description': user.description,
                                  'location': user.location
                                 }
        for tweet in response.data:
        # For each tweet, find the author's information
            author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
            result.append({'author_id': tweet.author_id, 
                           'username': author_info['username'],
                           'author_followers': author_info['followers'],
                           'author_tweets': author_info['tweets'],
                           'author_description': author_info['description'],
                           'author_location': author_info['location'],
                           'text': tweet.text,
                           'created_at': tweet.created_at,
                           'retweets': tweet.public_metrics['retweet_count'],
                           'replies': tweet.public_metrics['reply_count'],
                           'likes': tweet.public_metrics['like_count'],
                           'quote_count': tweet.public_metrics['quote_count']
                          })
    except:
        print('error')

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [ ]:
#store in csv files 
df.to_csv('July2220.csv', index=False)